In [1]:
import pandas as pd

In [3]:
df=pd.read_csv("data_formed.csv")

In [3]:
df

,Number of employees Last avail. yr,Cash & Cash Equivalents at the End of Period th USD Last avail. yr,Cash & Cash Equivalents at the End of Period th USD Year - 1,Cash & Cash Equivalents at the End of Period th USD Year - 2,Cash & Cash Equivalents at the End of Period th USD Year - 3,Cash & Cash Equivalents at the End of Period th USD Year - 4,Cash & Cash Equivalents at the End of Period th USD Year - 5,Cash & Cash Equivalents at the Beginning of Period th USD Last avail. yr,Cash & Cash Equivalents at the Beginning of Period th USD Year - 1,Cash & Cash Equivalents at the Beginning of Period th USD Year - 2,...,Ordinary dividends th USD Year - 3.4,Ordinary dividends th USD Year - 4.4,Ordinary dividends th USD Year - 5.4,Dividend share capital other th USD Last avail. yr.4,Dividend share capital other th USD Year - 1.4,Dividend share capital other th USD Year - 2.4,Dividend share capital other th USD Year - 3.4,Dividend share capital other th USD Year - 4.4,Dividend share capital other th USD Year - 5.4,Moody's Long-Term Rating
0,2100000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6048000.0,6102000.0,6124000.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0
1,70496.0,60279204.0,79887736.0,55261870.0,47388268.0,48840536.0,21664534.0,79887736.0,55261870.0,47388268.0,...,3936000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0
2,1541000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0
3,385691.0,17046024.0,13398674.0,8650064.0,16331826.0,17387124.0,17909580.0,13744663.0,9248496.0,16019321.0,...,7420108.0,9296486.0,4337823.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0
4,62000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,14652000.0,13798000.0,13001000.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1079,2256.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0
1080,5000.0,427118.0,354850.0,851012.0,1069214.0,110865.0,116149.0,325161.0,932232.0,1061020.0,...,121722.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0
1081,15018.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1236900.0,1082300.0,456100.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0
1082,30106.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,17300.0,17100.0,16900.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0


In [4]:
merged_df=df.drop("Moody's Long-Term Rating",axis=1)
y=df["Moody's Long-Term Rating"]

In [5]:
learn_x=merged_df[:900].values
learn_y=y[:900].values
test_x=merged_df[900:].values
test_y=y[900:].values

In [6]:
test_x.shape

(184, 3673)

In [7]:
test_y.shape

(184,)

In [8]:
from sklearn.preprocessing import StandardScaler

# DataFrameの全体を正規化
scaler = StandardScaler()
merged_df = pd.DataFrame(scaler.fit_transform(merged_df), columns=merged_df.columns)

In [9]:
from sklearn.decomposition import PCA

pca = PCA()
pca.fit(merged_df)
n_components = 100 # 削減後の次元数
pca = PCA(n_components=n_components)
pca_df = pd.DataFrame(pca.fit_transform(merged_df))

In [10]:
learn_x=pca_df[:900].values
learn_y=y[:900].values
test_x=pca_df[900:].values
test_y=y[900:].values

In [11]:
test_x.shape

(184, 100)

In [12]:

test_y.shape

(184,)

In [13]:
#成型官僚
#tensorflowで実験

In [15]:
import lightgbm as lgb

In [16]:
import numpy as np

In [18]:
from sklearn.model_selection import KFold

# ハイパーパラメータ
k = 5 # k-fold CVの分割数
num_boost_round = 200 # boostingの回数
params = {
'objective': 'regression',
'metric': 'mae',
'num_leaves': 128,
'learning_rate': 0.02,
'feature_fraction': 0.9,
'bagging_fraction': 0.8,
'bagging_freq': 5,
'verbose': -1,
'random_state': 1236
}
# CVを行うためのインデックスを作成
kf = KFold(n_splits=k, shuffle=True, random_state=1234)

# CVのループ
mae_list = []
for train_index, val_index in kf.split(learn_x):
    # train/valデータを作成
    train_x, train_y = learn_x[train_index], learn_y[train_index]
    val_x, val_y = learn_x[val_index], learn_y[val_index]
    dtrain = lgb.Dataset(train_x, label=train_y)
    dval = lgb.Dataset(val_x, label=val_y)
    # コールバック関数
    def print_progress(model, metric):
        print(f'[{model.current_iteration()}] {metric} : {model.best_score["valid_1"][metric]:.4f}')

    # モデルを構築
    model = lgb.train(params, dtrain, num_boost_round=num_boost_round, valid_sets=[dtrain, dval], callbacks=[lgb.callback.print_evaluation(period=20, show_stdv=False), lgb.callback.early_stopping(stopping_rounds=20)], verbose_eval=False)

    # テストデータで評価
    preds = model.predict(test_x)
    test_mae = np.mean(np.abs(test_y - preds))
    mae_list.append(test_mae)

# 結果を出力
print('Test MAE:', np.mean(mae_list))

Training until validation scores don't improve for 20 rounds
[20]	training's l1: 1.80404	valid_1's l1: 1.61448
[40]	training's l1: 1.52771	valid_1's l1: 1.49929
[60]	training's l1: 1.32831	valid_1's l1: 1.41899
[80]	training's l1: 1.17897	valid_1's l1: 1.38993
[100]	training's l1: 1.05345	valid_1's l1: 1.35311
[120]	training's l1: 0.949396	valid_1's l1: 1.3187
[140]	training's l1: 0.8613	valid_1's l1: 1.29598
[160]	training's l1: 0.795135	valid_1's l1: 1.27604
[180]	training's l1: 0.733599	valid_1's l1: 1.26219
[200]	training's l1: 0.678196	valid_1's l1: 1.25624
Did not meet early stopping. Best iteration is:
[200]	training's l1: 0.678196	valid_1's l1: 1.25624
Training until validation scores don't improve for 20 rounds
[20]	training's l1: 1.72538	valid_1's l1: 1.96091
[40]	training's l1: 1.4706	valid_1's l1: 1.81405
[60]	training's l1: 1.28148	valid_1's l1: 1.71232
[80]	training's l1: 1.12814	valid_1's l1: 1.64791
[100]	training's l1: 1.00934	valid_1's l1: 1.61252
[120]	training's l1:

In [20]:
model.save_model("lgbmodel")

In [36]:
predictions = model.predict(test_x)
for i in range (len(test_y)):
    print(predictions[i][0].round(),test_y[i])

13.0 14.0
18.0 18.0
15.0 15.0
15.0 16.0
14.0 14.0
14.0 14.0
15.0 15.0
12.0 12.0
13.0 13.0
15.0 14.0
11.0 10.0
15.0 11.0
15.0 13.0
8.0 9.0
14.0 14.0
15.0 16.0
13.0 12.0
15.0 16.0
15.0 14.0
13.0 13.0
13.0 13.0
11.0 11.0
11.0 11.0
11.0 11.0
13.0 13.0
12.0 12.0
12.0 15.0
4.0 2.0
15.0 17.25
15.0 15.0
13.0 12.0
14.0 13.0
13.0 13.0
10.0 9.0
9.0 11.0
10.0 13.0
15.0 15.0
13.0 12.0
13.0 12.0
14.0 14.0
13.0 13.0
11.0 10.0
15.0 14.0
8.0 8.0
13.0 14.0
13.0 13.0
13.0 13.0
15.0 14.0
13.0 14.0
17.0 17.0
11.0 11.0
16.0 16.0
12.0 13.0
14.0 13.0
15.0 15.0
13.0 12.0
11.0 12.0
10.0 15.0
14.0 14.0
14.0 14.0
13.0 14.0
15.0 15.0
16.0 16.0
13.0 14.0
13.0 13.0
14.0 14.0
14.0 15.0
14.0 14.0
16.0 16.0
21.0 21.0
13.0 13.0
13.0 13.0
12.0 13.25
12.0 12.0
11.0 11.0
15.0 17.0
12.0 12.0
14.0 15.0
11.0 13.0
7.0 7.0
12.0 12.0
11.0 12.0
4.0 4.0
14.0 14.0
14.0 12.0
13.0 13.0
13.0 14.0
13.0 13.0
9.0 9.0
6.0 6.0
7.0 7.0
16.0 16.0
3.0 2.0
7.0 7.0
16.0 14.0
21.0 21.0
15.0 14.0
15.0 15.0
14.0 13.0
14.0 14.0
15.0 14.0
15.0 14.0


In [30]:
test_y

array([14.  , 18.  , 15.  , 16.  , 14.  , 14.  , 15.  , 12.  , 13.  ,
       14.  , 10.  , 11.  , 13.  ,  9.  , 14.  , 16.  , 12.  , 16.  ,
       14.  , 13.  , 13.  , 11.  , 11.  , 11.  , 13.  , 12.  , 15.  ,
        2.  , 17.25, 15.  , 12.  , 13.  , 13.  ,  9.  , 11.  , 13.  ,
       15.  , 12.  , 12.  , 14.  , 13.  , 10.  , 14.  ,  8.  , 14.  ,
       13.  , 13.  , 14.  , 14.  , 17.  , 11.  , 16.  , 13.  , 13.  ,
       15.  , 12.  , 12.  , 15.  , 14.  , 14.  , 14.  , 15.  , 16.  ,
       14.  , 13.  , 14.  , 15.  , 14.  , 16.  , 21.  , 13.  , 13.  ,
       13.25, 12.  , 11.  , 17.  , 12.  , 15.  , 13.  ,  7.  , 12.  ,
       12.  ,  4.  , 14.  , 12.  , 13.  , 14.  , 13.  ,  9.  ,  6.  ,
        7.  , 16.  ,  2.  ,  7.  , 14.  , 21.  , 14.  , 15.  , 13.  ,
       14.  , 14.  , 14.  , 13.  , 15.  , 14.  , 14.  , 15.  , 17.  ,
       15.  , 15.  , 12.  , 18.  , 13.  ,  9.  ,  7.  , 14.  , 14.  ,
       13.  ,  9.  , 12.  , 12.  ,  5.  , 10.  , 10.  , 17.  , 10.  ,
       11.  , 13.  ,

In [ ]:
#boosting num
#200,1.284

In [21]:

import joblib
# PCAモデルの保存
joblib.dump(pca, 'pca_model100.pkl')

['pca_model100.pkl']